<a href="https://colab.research.google.com/github/Praveena-Krishnan/MY-GROCERY-Product-recommendation-system/blob/main/MyGrocery_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the necessary libraries
import numpy as np
import pandas as pd
import scipy.stats as stats

from difflib import get_close_matches



#visulisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#text handeling libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

#loading the dataset
df=pd.read_csv("/content/drive/MyDrive/BigBasket Products.csv",index_col='index')

df.head()
df.shape
df.isnull().sum()

#handleing null values
null_count=df.isnull().sum().sum()
total_count=np.product(df.shape)
print("{:.2f}".format(null_count/total_count*100))

#drop the null valued rows
df=df.dropna()
df.isnull().sum()

counts=df['category'].value_counts()
counts_df=pd.DataFrame({'Category':counts.index,'Counts':counts.values})

px.bar(data_frame=counts_df,
       x="Category",
       y="Counts",
       color="Counts",
       color_continuous_scale="blues",
       text_auto=True,
       title=f'count of items in each category')

counts=df['sub_category'].value_counts()
counts_df_1=pd.DataFrame({'Category':counts.index,'Counts':counts.values})[:10]

px.bar(data_frame=counts_df_1,
       x='Category',
       y='Counts',
       color='Counts',
       color_continuous_scale='blues',
       text_auto=True,
       title=f'Top 10 Bought sub-categories')

counts=df['brand'].value_counts()
counts_df_brand=pd.DataFrame({'Brand Name':counts.index,'Counts':counts.values})[:10]

px.bar(data_frame=counts_df_brand,
       x='Brand Name',
       y='Counts',
       color='Counts',
       color_continuous_scale='blues',
       text_auto=True,
       title=f'Top 10 Brand Items based on Item Counts')

counts=df['type'].value_counts()
counts_df_type=pd.DataFrame({'Type':counts.index,'Counts':counts.values})[:10]

3.53


In [2]:
#handleing null values
null_count=df.isnull().sum().sum()
total_count=np.product(df.shape)
print("{:.2f}".format(null_count/total_count*100))

0.00


now we are dropping the null value columns because they are only 3.54% in overall

In [ ]:
#drop the null valued rows
df=df.dropna()
df.isnull().sum()

product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
dtype: int64

In [ ]:
df.shape

(18840, 9)

so now we have 12925 products to build our recomendation


EPLORATORY DATA **ANALYSIS**

In [ ]:
counts=df['category'].value_counts()
counts_df=pd.DataFrame({'Category':counts.index,'Counts':counts.values})
counts_df

,Category,Counts
0,Beauty & Hygiene,5460
1,"Kitchen, Garden & Pets",2494
2,Snacks & Branded Foods,2468
3,Gourmet & World Food,2364
4,"Foodgrains, Oil & Masala",2173
5,Cleaning & Household,2091
6,"Bakery, Cakes & Dairy",665
7,Beverages,630
8,Baby Care,495


In [ ]:
px.bar(data_frame=counts_df,
       x="Category",
       y="Counts",
       color="Counts",
       color_continuous_scale="blues",
       text_auto=True,
       title=f'count of items in each category')

In [ ]:
counts=df['sub_category'].value_counts()
counts_df_1=pd.DataFrame({'sub_category':counts.index,'Counts':counts.values})[:10]
counts_df_1

,sub_category,Counts
0,Skin Care,1641
1,Hair Care,818
2,Bath & Hand Wash,808
3,Masalas & Spices,764
4,Storage & Accessories,658
5,Men's Grooming,649
6,Fragrances & Deos,627
7,Crockery & Cutlery,621
8,Ready To Cook & Eat,557
9,Organic Staples,550


In [ ]:
px.bar(data_frame=counts_df_1,
       x='sub-cateogry',
       y='Counts',
       color='Counts',
       color_continuous_scale='blues',
       text_auto=True,
       title=f'Top 10 Bought sub-categories')

In [ ]:
counts=df['brand'].value_counts()
counts_df_brand=pd.DataFrame({'Brand Name':counts.index,'Counts':counts.values})[:10]
counts_df_brand

,Brand Name,Counts
0,bb Royal,278
1,BB Home,172
2,Amul,153
3,Himalaya,139
4,Cello,104
5,BIOTIQUE,103
6,DP,101
7,Keya,101
8,Organic Tattva,99
9,MTR,97


In [ ]:
px.bar(data_frame=counts_df_brand,
       x='Brand Name',
       y='Counts',
       color='Counts',
       color_continuous_scale='blues',
       text_auto=True,
       title=f'Top 10 Brand Items based on Item Counts')

In [ ]:
counts=df['type'].value_counts()
counts_df_type=pd.DataFrame({'Type':counts.index,'Counts':counts.values})[:10]
counts_df_type

In [ ]:
px.bar(data_frame=counts_df_type,
       x='Type',
       y='Counts',
       color='Counts',
       color_continuous_scale='blues',
       text_auto=True,
       title=f'Top 10 Types of Products based on Item Counts')

Rating Analysis

In [ ]:
print(df['rating'].describe())
df['rating'].hist(bins=10)

In [ ]:
pd.cut(df.rating,bins=[0,1,2,3,4,5]).reset_index().groupby(['rating']).size()

Feature Engineering

In [ ]:
df['discount']=(df['market_price']-df['sale_price'])*100/df['market_price']
print(df['discount'].describe())
pd.cut(df.discount,bins = [-1,0,10,20,30,40,50,60,80,90,100]).reset_index().groupby(['discount']).size()

In [ ]:
df['discount'].hist()

checking the relation between rating and discount using bivariate analysis

In [ ]:
ax = df.plot.scatter(x='rating',y='discount')

cleaning the string values

In [ ]:
df2 = df.copy()
rmv_spc = lambda a:a.strip()
get_list = lambda a:list(map(rmv_spc,re.split('& |, |\*|n', a)))
for col in ['category', 'sub_category', 'type']:
    df2[col] = df2[col].apply(get_list)
def cleaner(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
for col in ['category', 'sub_category', 'type','brand']:
    df2[col] = df2[col].apply(cleaner)
def couple(x):
    return ' '.join(x['category']) + ' ' + ' '.join(x['sub_category']) + ' '+x['brand']+' ' +' '.join( x['type'])
df2['product_classification_features'] = df2.apply(couple, axis=1)
